In [139]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ratings/ratings_small.csv


In [140]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt  


In [141]:
review = pd.read_csv('../input/ratings/ratings_small.csv')
review.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [142]:
review = review.drop(['timestamp'],axis=1)//TIMESTAMP IS ALREADY DROPPED 

SyntaxError: invalid syntax (<ipython-input-142-ca5d6b0b7c5a>, line 1)

In [ ]:
review.shape

In [ ]:
review.head()

In [ ]:
review.info()

In [ ]:
#Total users and total movies in the data set 
review['userId'].nunique()

In [ ]:
review['movieId'].nunique()

In [ ]:
#to find the interactions using Histogram
inter = review.groupby('userId').count()['movieId']
plt.hist(inter, bins=30)
plt.show()

In [143]:
#user interaction matrix 

In [144]:
inter_matrix = review.pivot(index='userId',columns='movieId',values='rating')

In [145]:
inter_matrix.shape

(671, 9066)

In [146]:
inter_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
#replace the missing values with Zeros
inter_matrix = inter_matrix.fillna(0)

In [148]:
inter_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
#finding the similarity 

from sklearn.metrics.pairwise import cosine_similarity

def similar_users(user_id,inter_matrix):
    similarity = []
    for user in range(1,inter_matrix.shape[0]+1):
        sim = cosine_similarity([inter_matrix.loc[user_id]],[inter_matrix.loc[user]])
        similarity.append(user)
        similarity.append(sim)
        
      
        
    
    return similarity
   
    
    most_similar_users = [tup[0] for tup in similarity]
    
    similarity_score = [tup[1] for tup in similarity ]
    
    #same should be removed
    
    most_similar_users.remove(user_id)
    
    similarity_score.remove(similarity_score[0])
        
    return most_similar_users,similarity_score


In [201]:
similar_users(564, inter_matrix)[1]

array([[0.08167271]])

In [212]:
def recom(user_id,num_of_movies,user_item_inter):
    
    most_similar_users = similar_users(user_id,inter_matrix)[0]
    
    movie_ids = set(list(inter_matrix.columns[np.where(inter_matrix.loc[user_id] > 0)]))
    
    recom = []
    
    already_inter = movie_ids.copy()
    
    for similar_user in most_similar_users:
        
        if len(recom) < num_of_movies:
            
            similar_users_movie_ids = set(list(inter_matrix.columns[np.where(inter_matrix.loc[similar_user] > 0)]))
            
            recom.extend(list(similar_users_movie_ids.difference(already_inter)))
            
            already_inter = already_inter.union(similar_users_movie_ids)
        else:
            break 
            
    return recom[:num_of_movies]

In [214]:
similar_users(564, inter_matrix)[0]
    

array([[0.]])